In [3]:

import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import talib
from ta import add_all_ta_features
from ta.utils import dropna
import matplotlib.pyplot as plt

# Function to fetch stock data
def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Function to calculate all technical indicators
def calculate_indicators(df):
    # ADX
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    # MACD
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
    # Hull Moving Average
    def hull_moving_average(df, period):
        wma_half = df['Close'].rolling(window=int(period/2)).apply(lambda x: ((np.arange(0, int(period/2)) + 1)*x).sum()/(np.arange(0, int(period/2)) + 1).sum(), raw=True)
        wma_full = df['Close'].rolling(window=period).apply(lambda x: ((np.arange(0, period) + 1)*x).sum()/(np.arange(0, period) + 1).sum(), raw=True)
        hma = 2 * wma_half - wma_full
        hma = hma.rolling(window=int(np.sqrt(period))).mean()
        return hma
    df['HMA_9'] = hull_moving_average(df, 9)
    
    # Ichimoku Cloud
    tenkan_period = 9
    kijun_period = 26
    senkou_span_b_period = 52
    df['Tenkan_Sen'] = (df['High'].rolling(window=tenkan_period).max() + df['Low'].rolling(window=tenkan_period).min()) / 2
    df['Kijun_Sen'] = (df['High'].rolling(window=kijun_period).max() + df['Low'].rolling(window=kijun_period).min()) / 2
    df['Senkou_Span_A'] = ((df['Tenkan_Sen'] + df['Kijun_Sen']) / 2).shift(kijun_period)
    df['Senkou_Span_B'] = ((df['High'].rolling(window=senkou_span_b_period).max() + df['Low'].rolling(window=senkou_span_b_period).min()) / 2).shift(kijun_period)
    
    # Aroon Indicator
    df['Aroon_Up'], df['Aroon_Down'] = talib.AROON(df['High'], df['Low'], timeperiod=14)
    
    # Guppy MMA (using multiple EMAs)
    periods = [3, 5, 8, 10, 12, 15, 30, 35, 40, 45, 50, 60]
    for period in periods:
        df[f'EMA_{period}'] = talib.EMA(df['Close'], timeperiod=period)
    
    # RSI
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    
    # Stochastic RSI
    def stoch_rsi(df, period=14, smoothK=3, smoothD=3):
        rsi = talib.RSI(df['Close'], timeperiod=period)
        stochrsi = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
        stochrsi_K = stochrsi.rolling(smoothK).mean()
        stochrsi_D = stochrsi_K.rolling(smoothD).mean()
        return stochrsi_K, stochrsi_D
    df['Stoch_RSI_K'], df['Stoch_RSI_D'] = stoch_rsi(df)
    
    # Momentum
    df['Momentum'] = talib.MOM(df['Close'], timeperiod=10)
    
    # ROC
    df['ROC'] = talib.ROC(df['Close'], timeperiod=10)
    
    # TRIX
    df['TRIX'] = talib.TRIX(df['Close'], timeperiod=15)
    
    # TSI
    def tsi(df, long=25, short=13):
        diff = df['Close'].diff(1)
        abs_diff = diff.abs()
        
        ema_diff = diff.ewm(span=short, adjust=False).mean()
        ema_abs_diff = abs_diff.ewm(span=short, adjust=False).mean()
        
        ema_ema_diff = ema_diff.ewm(span=long, adjust=False).mean()
        ema_ema_abs_diff = ema_abs_diff.ewm(span=long, adjust=False).mean()
        
        tsi = 100 * (ema_ema_diff / ema_ema_abs_diff)
        return tsi
    df['TSI'] = tsi(df)
    
    # Connors RSI
    def connors_rsi(df, rsi_period=3, streak_period=2, pct_rank_period=100):
        # RSI
        rsi = talib.RSI(df['Close'], timeperiod=rsi_period)
        
        # Up/Down Streak
        streak = np.where(df['Close'].diff() > 0, 1, np.where(df['Close'].diff() < 0, -1, 0))
        streak_rsi = talib.RSI(pd.Series(streak, index=df.index), timeperiod=streak_period)
        
        # Percent Rank
        pct_rank = df['Close'].rolling(pct_rank_period).apply(lambda x: (x[-1] > x).mean())
        
        # Connors RSI
        connors = (rsi + streak_rsi + pct_rank) / 3
        return connors
    df['Connors_RSI'] = connors_rsi(df)
    
    # Fisher Transform
    def fisher_transform(df, period=10):
        high, low = df['High'], df['Low']
        median_price = (high + low) / 2
        max_high = high.rolling(period).max()
        min_low = low.rolling(period).min()
        
        normalized = (2 * ((median_price - min_low) / (max_high - min_low))) - 1
        normalized = normalized.clip(-0.999, 0.999)
        
        fisher = 0.5 * np.log((1 + normalized) / (1 - normalized))
        return fisher
    df['Fisher_Transform'] = fisher_transform(df)
    
    # OBV
    df['OBV'] = talib.OBV(df['Close'], df['Volume'])
    
    # Accumulation/Distribution
    df['ADL'] = talib.AD(df['High'], df['Low'], df['Close'], df['Volume'])
    
    # MFI
    df['MFI'] = talib.MFI(df['High'], df['Low'], df['Close'], df['Volume'], timeperiod=14)
    
    # CMF
    def cmf(df, period=20):
        mfv = ((df['Close'] - df['Low']) - (df['High'] - df['Close'])) / (df['High'] - df['Low'])
        mfv = mfv * df['Volume']
        cmf = mfv.rolling(period).sum() / df['Volume'].rolling(period).sum()
        return cmf
    df['CMF'] = cmf(df)
    
    # VWAP
    def calculate_vwap(df):
        vwap = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()
        return vwap
    df['VWAP'] = calculate_vwap(df)
    
    # VWMA
    def vwma(df, period=20):
        vwma = (df['Close'] * df['Volume']).rolling(period).sum() / df['Volume'].rolling(period).sum()
        return vwma
    df['VWMA_20'] = vwma(df)
    
    # ATR
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    # Bollinger Bands
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = talib.BBANDS(df['Close'], timeperiod=20)
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_%B'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # Choppiness Index
    def choppiness_index(df, period=14):
        atr_sum = df['ATR'].rolling(period).sum()
        high_max = df['High'].rolling(period).max()
        low_min = df['Low'].rolling(period).min()
        ci = 100 * np.log10(atr_sum / (high_max - low_min)) / np.log10(period)
        return ci
    df['Choppiness_Index'] = choppiness_index(df)
    
    # RVI
    def rvi(df, period=14):
        close_diff = df['Close'].diff()
        high_diff = df['High'].diff()
        low_diff = df['Low'].diff()
        
        num = (close_diff + 2 * high_diff + 2 * low_diff).rolling(period).sum()
        den = (4 * high_diff.abs() + 4 * low_diff.abs()).rolling(period).sum()
        rvi = num / den
        return rvi
    df['RVI'] = rvi(df)
    
    # Historical Volatility
    df['Historical_Volatility'] = df['Close'].pct_change().rolling(20).std() * np.sqrt(252)
    
    # Standard Deviation
    df['Std_Dev_20'] = df['Close'].rolling(20).std()
    
    # Donchian Channel
    df['Donchian_High'] = df['High'].rolling(20).max()
    df['Donchian_Low'] = df['Low'].rolling(20).min()
    df['Donchian_Middle'] = (df['Donchian_High'] + df['Donchian_Low']) / 2
    
    # Pivot Points
    def calculate_pivot_points(df):
        prev_day = df.shift(1)
        df['Pivot'] = (prev_day['High'] + prev_day['Low'] + prev_day['Close']) / 3
        df['R1'] = 2 * df['Pivot'] - prev_day['Low']
        df['S1'] = 2 * df['Pivot'] - prev_day['High']
        df['R2'] = df['Pivot'] + (prev_day['High'] - prev_day['Low'])
        df['S2'] = df['Pivot'] - (prev_day['High'] - prev_day['Low'])
        df['R3'] = df['Pivot'] + 2 * (prev_day['High'] - prev_day['Low'])
        df['S3'] = df['Pivot'] - 2 * (prev_day['High'] - prev_day['Low'])
        return df
    df = calculate_pivot_points(df)
    
    # Fibonacci Levels
    def calculate_fib_levels(df):
        prev_day = df.shift(1)
        high = prev_day['High'].iloc[-1]
        low = prev_day['Low'].iloc[-1]
        df['Fib_0'] = high
        df['Fib_0.236'] = high - (high - low) * 0.236
        df['Fib_0.382'] = high - (high - low) * 0.382
        df['Fib_0.5'] = high - (high - low) * 0.5
        df['Fib_0.618'] = high - (high - low) * 0.618
        df['Fib_1'] = low
        return df
    df = calculate_fib_levels(df)
    
    # Darvas Box
    def darvas_box(df, period=5):
        df['Darvas_High'] = df['High'].rolling(period).max()
        df['Darvas_Low'] = df['Low'].rolling(period).min()
        return df
    df = darvas_box(df)
    
    # William Fractal
    def william_fractal(df):
        # Bullish Fractal
        fractal_bull = (df['High'] > df['High'].shift(1)) & \
                      (df['High'] > df['High'].shift(2)) & \
                      (df['High'] > df['High'].shift(-1)) & \
                      (df['High'] > df['High'].shift(-2))
        
        # Bearish Fractal
        fractal_bear = (df['Low'] < df['Low'].shift(1)) & \
                       (df['Low'] < df['Low'].shift(2)) & \
                       (df['Low'] < df['Low'].shift(-1)) & \
                       (df['Low'] < df['Low'].shift(-2))
        
        df['Fractal_Bull'] = fractal_bull.astype(int)
        df['Fractal_Bear'] = fractal_bear.astype(int)
        return df
    df = william_fractal(df)
    
    return df

# Function to create features and target
def prepare_data(df, forecast_days=15):
    # Create target (future price)
    df['Target'] = df['Close'].shift(-forecast_days)
    
    # Drop rows with NaN values
    df = df.dropna()
    
    # Separate features and target
    X = df.drop(['Target'], axis=1)
    y = df['Target']
    
    return X, y

# Function to train model
def train_model(X, y):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Scale features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    # Evaluate model
    train_pred = model.predict(X_train_scaled)
    test_pred = model.predict(X_test_scaled)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    
    print(f"Train RMSE: {train_rmse:.2f}")
    print(f"Test RMSE: {test_rmse:.2f}")
    
    return model, scaler

# Function to make predictions
def predict_future(model, scaler, df, forecast_days=15):
    # Use the most recent data point
    last_data = df.iloc[-1:].copy()
    
    # Prepare features for prediction (without target)
    X_pred = last_data.drop(['Target'], axis=1, errors='ignore')
    X_pred_scaled = scaler.transform(X_pred)
    
    # Make prediction
    prediction = model.predict(X_pred_scaled)[0]
    
    return prediction

# Main function
def main():
    # Get ticker symbol from user
    ticker = input("Enter stock ticker (e.g., AAPL): ").upper()
    
    # Define date range (last 5 years)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5*365)
    
    # Get stock data
    print(f"Fetching data for {ticker}...")
    stock_data = get_stock_data(ticker, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
    
    if stock_data.empty:
        print("No data found for this ticker.")
        return
    
    # Calculate all indicators
    print("Calculating technical indicators...")
    stock_data = calculate_indicators(stock_data)
    
    # Prepare data for modeling
    X, y = prepare_data(stock_data)
    
    # Train model
    print("Training model...")
    model, scaler = train_model(X, y)
    
    # Make predictions for next 15 days
    print("\n15-day price prediction:")
    predictions = []
    current_data = stock_data.copy()
    
    for day in range(1, 16):
        # Predict next day
        next_price = predict_future(model, scaler, current_data)
        predictions.append(next_price)
        
        # Create a new row for the predicted day
        new_row = current_data.iloc[-1:].copy()
        new_row.index = [new_row.index[0] + timedelta(days=1)]
        new_row['Close'] = next_price
        
        # Update some basic values (simplistic approach)
        new_row['Open'] = current_data['Close'].iloc[-1]
        new_row['High'] = max(new_row['Open'], next_price)
        new_row['Low'] = min(new_row['Open'], next_price)
        new_row['Volume'] = current_data['Volume'].rolling(5).mean().iloc[-1]  # Use average volume
        
        # Append to dataframe
        current_data = pd.concat([current_data, new_row])
        
        # Recalculate indicators for the new row
        current_data = calculate_indicators(current_data)
        
        # Print prediction
        print(f"Day {day}: {next_price:.2f}")
    
    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(stock_data.index[-60:], stock_data['Close'][-60:], label='Historical Prices')
    future_dates = [stock_data.index[-1] + timedelta(days=i) for i in range(1, 16)]
    plt.plot(future_dates, predictions, 'r-', label='Predicted Prices')
    plt.title(f'{ticker} 15-Day Price Prediction')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid()
    plt.show()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'talib'